In [ ]:
#imports
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt
import glob
import tensorflow as tf
import datetime

random.seed(108)

In [ ]:
#Classes for data 
class NumberOfBus:
    def __init__(self, filename, size, bndbox):
        self.filename = filename
        self.size = Size(size)
        self.bndbox = BndBox(bndbox)

    @staticmethod
    def extract_fields_from_xml(xml_file):
        root = ET.parse(xml_file).getroot()
        filename = ''
        size = ''
        bbox = ''
        # Parse the XML Tree
        for elem in root:
            # Get the file name 
            if elem.tag == "filename":
                filename = elem.text
                
            # Get the image size
            elif elem.tag == "size":
                size = elem

            # Get details of the bounding box 
            elif elem.tag == "object":
                for subelem in elem:
                    if subelem.tag == "bndbox":
                        bbox = subelem
        return NumberOfBus(filename, size, bbox)

class Size:
    def __init__(self, xmlNode):
        width = 0
        height = 0
        depth = 0
        for node in xmlNode:
            if node.tag == "width":
                width = int(node.text)
            elif node.tag == "height":
                height = int(node.text)
            elif node.tag == "depth":
                depth = int(node.text)
        self.width = width
        self.height = height
        self.depth  = depth

class BndBox:
    def __init__(self, xmlNode) -> None:
        xmax = 0
        xmin = 0
        ymin = 0
        ymax = 0
        for node in xmlNode:
            if node.tag == "xmax":
                xmax = int(node.text)
            elif node.tag == "xmin":
                xmin = int(node.text)
            elif node.tag == "ymin":
                ymin = int(node.text)
            elif node.tag == "ymax":
                ymax = int(node.text)
        self.xmax = xmax
        self.xmin = xmin
        self.ymin = ymin
        self.ymax = ymax 

In [ ]:
xml_file = ('DataXml\\test\\image-1-53_jpg.rf.acf7c6c8961a2e16284a42187fa13aff.xml')
obj = NumberOfBus.extract_fields_from_xml(xml_file)

In [ ]:
os.listdir("DataXml\\test")
files = glob.glob(os.path.join('DataXml\\test', '*.xml'))
list_bus_data = []
for filePath in files: 
    list_bus_data.append(NumberOfBus.extract_fields_from_xml(filePath))
print(list_bus_data)
print(list_bus_data[1].bndbox.xmin)

In [ ]:
# Convert the info dict to the required yolo format and write it to disk
def convert_to_yolov5(filePath, busData: list):
    # For each bounding box
    for busItem in busData:
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        print_buffer = []
        b_center_x = (busItem.bndbox.xmin + busItem.bndbox.xmax) / 2 
        b_center_y = (busItem.bndbox.ymin + busItem.bndbox.ymax) / 2
        b_width    = (busItem.bndbox.xmax - busItem.bndbox.xmin)
        b_height   = (busItem.bndbox.ymax - busItem.bndbox.ymin)
        
        # Normalise the co-ordinates by the dimensions of the image
        b_center_x /= busItem.size.width
        b_center_y /= busItem.size.height
        b_width    /= busItem.size.width
        b_height   /= busItem.size.height 
        
        #Write the bbox details to the file 
        print_buffer.append("{:.3f} {:.3f} {:.3f} {:.3f}".format(b_center_x, b_center_y, b_width, b_height))
        
        # Name of the file which we have to save 
        filename = busItem.filename[0:len(busItem.filename)-3] + "txt"
        save_file_name = os.path.join(filePath, filename)
        # Save the annotation to disk 
        print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [ ]:
# Get the annotations
annotations = glob.glob(os.path.join('DataXml\\train', '*.xml'))
commonPath = os.path.join('Data', 'train')
filePath = os.path.join(commonPath, 'labels')
annotations.sort()
print(len(annotations))
# Convert and save the annotations
info_list = list()
for ann in tqdm(annotations):
    print(ann)
    info_list.append(NumberOfBus.extract_fields_from_xml(ann))
    convert_to_yolov5(filePath, info_list)
annotations = [os.path.join(filePath, x) for x in os.listdir(filePath) if x[-3:] == "txt"]

In [ ]:
images = glob.glob(os.path.join('DataXml\\train', '*.jpg'))
commonPath = os.path.join('Data', 'train')
filePath = os.path.join(commonPath, 'images')
print(filePath)
print(images)
for f in images:
    try:
        shutil.copy(f, filePath)
    except:
        print(f)

In [ ]:
# Get the annotations
annotations = glob.glob(os.path.join('DataXml\\test', '*.xml'))
commonPath = os.path.join('Data', 'test')
filePath = os.path.join(commonPath, 'labels')
annotations.sort()
print(len(annotations))
# Convert and save the annotations
info_list = list()
for ann in tqdm(annotations):
    print(ann)
    info_list.append(NumberOfBus.extract_fields_from_xml(ann))
    convert_to_yolov5(filePath, info_list)
annotations = [os.path.join(filePath, x) for x in os.listdir(filePath) if x[-3:] == "txt"]

In [ ]:
images = glob.glob(os.path.join('DataXml\\test', '*.jpg'))
commonPath = os.path.join('Data', 'test')
filePath = os.path.join(commonPath, 'images')
for f in images:
    try:
        shutil.move(f, filePath)
    except:
        print(f)
        assert False

In [ ]:
# Get the annotations
annotations = glob.glob(os.path.join('DataXml\\valid', '*.xml'))
commonPath = os.path.join('Data', 'valid')
filePath = os.path.join(commonPath, 'labels')
annotations.sort()
print(len(annotations))
# Convert and save the annotations
info_list = list()
for ann in tqdm(annotations):
    print(ann)
    info_list.append(NumberOfBus.extract_fields_from_xml(ann))
    convert_to_yolov5(filePath, info_list)
annotations = [os.path.join(filePath, x) for x in os.listdir(filePath) if x[-3:] == "txt"]

In [ ]:
images = glob.glob(os.path.join('DataXml\\valid', '*.jpg'))
commonPath = os.path.join('Data', 'valid')
filePath = os.path.join(commonPath, 'images')
print(filePath)
print(images)
for f in images:
    try:
        shutil.copy(f, filePath)
    except:
        print(f)

In [ ]:
print(len(annotations))
print(annotations)

In [ ]:
random.seed(0)

def plot_bounding_box(image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size
    
    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    print(transformed_annotations)
    transformed_annotations[:,[0,2]] = annotations[:,[0,2]] * w
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * h 
    
    transformed_annotations[:,0] = transformed_annotations[:,0] - (transformed_annotations[:,2] / 2)
    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,0] + transformed_annotations[:,2]
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    
    for ann in transformed_annotations:
        x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)))
    
    plt.imshow(np.array(image))
    plt.show()

# Get any random annotation file 
annotation_file = random.choice(annotations[:500:])
with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]
print(annotation_file)
print(annotation_list)
#Get the corresponding image file
image_file = annotation_file.replace("Data\\labels\\valid", "Data\\images\\valid")[0:len(annotation_file)-3]
image_file += "jpg"
print(image_file)
assert os.path.exists(image_file)

#Load the image
print(annotations)
image = Image.open(image_file)

#Plot the Bounding Box
plot_bounding_box(image, annotation_list)

In [ ]:
if not os.path.exists('yolov7'):
    !git clone https://github.com/WongKinYiu/yolov7.git

In [ ]:
!tensorboard --logdir runs/train

In [ ]:
!wandb login e478cc6c0a4343a9912bfe3a45254bbf4c6139c1

In [1]:
!python yolov7/train.py --workers 8 --device 0 --batch-size 8 --cfg cfg/training/yolov7-d6.yaml --epochs 100 --data Data/data.yaml --weights yolov7_training.pt --hyp data/hyp.scratch.custom.yaml

In [ ]:
!python yolov7/detect.py --source Data/test/images --weights yolov7_training.pt --conf 0.25 --name yolo_road_det

In [ ]:
detections_dir = "runs/detect/yolo_road_det4"
detection_images = [os.path.join(detections_dir, x) for x in os.listdir(detections_dir)]

random_detection_image = Image.open(random.choice(detection_images))
plt.imshow(np.array(random_detection_image))